In [1]:
import pandas as pd

## Introduction

The objective of this project is to create a prototype of a website that estimates the shortest distance needed to travel by car to multiple cities in California and back. The intended audience is anyone who is interested in going on a road trip through whichever California cities pique their interest, and possibly also go to some popular attractions.

The team collected data on the latitude and longitude of most cities in California, as well as the prevalence of violent crime in each city and the address(es) of any Chipotle stores in each city. Lastly, the team also found the latitude and longitude coordinates of ten popular attractions in the state.

The most important data were the coordinates, as those were used by the model (in the next notebook) to minimize distance. The purpose of the violent crime data is to help the users of the website know which cities are the safest to stay overnight at, and the Chipotle data gives users a starting point regarding where they can get food.

## Opening the DataFrames & Basic EDA

The following dataframe contains the name, latitude, and longitude of cities in California.

It was extracted from https://www.kaggle.com/datasets/camnugent/california-housing-feature-engineering/data.

In [2]:
california_cities = pd.read_csv('cal_cities_lat_long.csv')

In [3]:
california_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       459 non-null    object 
 1   Latitude   459 non-null    float64
 2   Longitude  459 non-null    float64
dtypes: float64(2), object(1)
memory usage: 10.9+ KB


In [4]:
california_cities.columns = ['city', 'latitude', 'longitude']

In [5]:
california_cities.head(10)

,city,latitude,longitude
0,Adelanto,34.582769,-117.409214
1,Agoura Hills,34.153339,-118.761675
2,Alameda,37.765206,-122.241636
3,Albany,37.886869,-122.297747
4,Alhambra,34.095286,-118.127014
5,Aliso Viejo,33.575000,-117.725556
6,Alturas,41.487114,-120.542456
7,Amador City,38.419356,-120.824103
8,American Canyon,38.174917,-122.260803
9,Anaheim,33.835292,-117.914503


The following dataframe contains the city, state, address, longitude, and latitude of all the Chipotle restaurants in the United States.

It was extracted from https://www.kaggle.com/datasets/jeffreybraun/chipotle-locations.

In [6]:
chipotle_stores = pd.read_csv('chipotle_stores.csv')

In [7]:
chipotle_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2629 entries, 0 to 2628
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   state      2629 non-null   object 
 1   location   2629 non-null   object 
 2   address    2629 non-null   object 
 3   latitude   2629 non-null   float64
 4   longitude  2629 non-null   float64
dtypes: float64(2), object(3)
memory usage: 102.8+ KB


For this project, the team only needs the data of the Chipotle stores in California.

In [8]:
ca_chipotle_stores = chipotle_stores[chipotle_stores['state'] == 'California'].copy()

ca_chipotle_stores.drop(columns = 'state', inplace = True)
ca_chipotle_stores.rename(columns = {'location':'city'}, inplace = True)
ca_chipotle_stores.reset_index(drop = True, inplace = True)

In [9]:
ca_chipotle_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   city       421 non-null    object 
 1   address    421 non-null    object 
 2   latitude   421 non-null    float64
 3   longitude  421 non-null    float64
dtypes: float64(2), object(2)
memory usage: 13.3+ KB


In [10]:
ca_chipotle_stores.head(10)

,city,address,latitude,longitude
0,Alameda,"2314 S Shore Ctr Alameda, CA 94501 US",37.756121,-122.250114
1,Alameda,"2610 5th St Ste A Alameda, CA 94501 US",37.786141,-122.281207
2,Alhambra,"2121 W Main St Ste 210 Alhambra, CA 91801 US",34.092555,-118.145842
3,Aliso Viejo,"26661 Aliso Creek Rd Ste A Aliso Viejo, CA 926...",33.576980,-117.725426
4,Anaheim,"2002 E Lincoln Ave Anaheim, CA 92806 US",33.839580,-117.888675
5,Anaheim,"5631 E La Palma Ave Anaheim, CA 92807 US",33.860570,-117.792283
6,Anaheim,"8182 E Santa Ana Canyon Rd Ste 195 Anaheim, CA...",33.868028,-117.748151
7,Anaheim,"927 S Euclid St Ste B Anaheim, CA 92802 US",33.818639,-117.941768
8,Apple Valley,"19179 Bear Valley Rd Ste 4 Apple Valley, CA 92...",34.470696,-117.245791
9,Arcadia,"400 S Baldwin Ave Ste FC-9 Arcadia, CA 91007 US",34.135727,-118.049737


I see no need for **US** to be written at the end of every **address** value. After all, anyone going on a road trip within California will know they are in the U.S.

In [11]:
ca_chipotle_stores['address'] = ca_chipotle_stores['address'].replace(' US$', '', regex = True)

How many Chipotle stores are there in each city? In the next line, I create a column that calculates exactly this.

In [12]:
ca_city_cmg_counts = ca_chipotle_stores.groupby('city').size().reset_index(name = 'cmg_stores')

ca_chipotle_stores = pd.merge(ca_chipotle_stores, ca_city_cmg_counts, on = 'city')

Personally, I think it makes sense for **cmg_stores** to be the second column.

In [13]:
ca_chipotle_stores_columns = list(ca_chipotle_stores.columns)

ca_chipotle_stores_columns.remove('cmg_stores')
ca_chipotle_stores_columns.insert(1, 'cmg_stores')

ca_chipotle_stores = ca_chipotle_stores[ca_chipotle_stores_columns]

In [14]:
ca_chipotle_stores.head(10)

,city,cmg_stores,address,latitude,longitude
0,Alameda,2,"2314 S Shore Ctr Alameda, CA 94501",37.756121,-122.250114
1,Alameda,2,"2610 5th St Ste A Alameda, CA 94501",37.786141,-122.281207
2,Alhambra,1,"2121 W Main St Ste 210 Alhambra, CA 91801",34.092555,-118.145842
3,Aliso Viejo,1,"26661 Aliso Creek Rd Ste A Aliso Viejo, CA 92656",33.576980,-117.725426
4,Anaheim,4,"2002 E Lincoln Ave Anaheim, CA 92806",33.839580,-117.888675
5,Anaheim,4,"5631 E La Palma Ave Anaheim, CA 92807",33.860570,-117.792283
6,Anaheim,4,"8182 E Santa Ana Canyon Rd Ste 195 Anaheim, CA...",33.868028,-117.748151
7,Anaheim,4,"927 S Euclid St Ste B Anaheim, CA 92802",33.818639,-117.941768
8,Apple Valley,1,"19179 Bear Valley Rd Ste 4 Apple Valley, CA 92308",34.470696,-117.245791
9,Arcadia,1,"400 S Baldwin Ave Ste FC-9 Arcadia, CA 91007",34.135727,-118.049737


Now let's create a dataframe that has only the unique **city** values and their corresponding **cmg_stores** values.

In [15]:
ca_city_cmg_counts_df = ca_chipotle_stores[['city', 'cmg_stores']].copy()
ca_city_cmg_counts_df = ca_city_cmg_counts_df.drop_duplicates()

I am curious to see the summary statistics of the **cmg_stores** column.

In [16]:
ca_city_cmg_counts_df['cmg_stores'].describe()

count    245.000000
mean       1.718367
std        2.017919
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       20.000000
Name: cmg_stores, dtype: float64

It appears that roughly half of cities in California do not have any Chipotle locations. Additionally, of those that do, at least 75% of them have only one or two.

According to the IQR outlier detection method, any **cmg_stores** values greater than 3.5 is a high outlier. Which cities are high outliers in this regard?

In [17]:
ca_city_cmg_counts_df[ca_city_cmg_counts_df['cmg_stores'] >= 4].sort_values(by = 'cmg_stores', ascending = False)

,city,cmg_stores
162,Los Angeles,20
304,San Diego,15
329,San Jose,14
287,Sacramento,12
320,San Francisco,9
13,Bakersfield,6
109,Fresno,5
136,Irvine,5
4,Anaheim,4
70,Culver City,4


I am not at all surprised by the cities that take up the top 5 spots; all 5 of them have large populations and are visited by tourists frequently.

The following dataframe contains the offenses known to law enforcement by city in California, 2019.

It was extracted from https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/tables/table-8/table-8-state-cuts/california.xls.

In [18]:
crime_by_city = pd.read_excel('known_offenses_2019.xls')

In [19]:
crime_by_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Table 8      462 non-null    object
 1   Unnamed: 1   458 non-null    object
 2   Unnamed: 2   458 non-null    object
 3   Unnamed: 3   458 non-null    object
 4   Unnamed: 4   458 non-null    object
 5   Unnamed: 5   458 non-null    object
 6   Unnamed: 6   458 non-null    object
 7   Unnamed: 7   458 non-null    object
 8   Unnamed: 8   458 non-null    object
 9   Unnamed: 9   458 non-null    object
 10  Unnamed: 10  458 non-null    object
 11  Unnamed: 11  458 non-null    object
dtypes: object(12)
memory usage: 43.4+ KB


Something isn't quite right with this dataframe, as evidenced by the fact that most of its columns are unnamed.

In [20]:
crime_by_city.head(10)

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,CALIFORNIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"by City, 2019",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson
4,Adelanto,34491,276,1,20,42,213,459,136,209,114,14
5,Agoura Hills,20490,21,0,6,4,11,306,66,223,17,0
6,Alameda,78907,162,0,7,94,61,2579,218,1958,403,29
7,Albany,20083,40,0,8,21,11,685,105,534,46,1
8,Alhambra,84837,161,2,11,89,59,1749,259,1303,187,8
9,Aliso Viejo,52247,27,1,3,13,10,433,57,351,25,1


The first three rows are unnecessary, and hence should be dropped. Furthermore, the fourth row should be the index. Lastly, I should convert the column names to snake_case.

In [21]:
crime_by_city.columns = crime_by_city.iloc[3]

crime_by_city = crime_by_city[4:]

crime_by_city.reset_index(drop = True, inplace = True)

crime_by_city.columns = ['city', 'population', 'violent_crime_count', 'murder_manslaughter', 'rape', 'robbery', 
                         'aggravated_assault', 'property_crime', 'burglary', 'larceny', 'car_theft', 'arson']

In [22]:
crime_by_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 458 non-null    object
 1   population           457 non-null    object
 2   violent_crime_count  457 non-null    object
 3   murder_manslaughter  457 non-null    object
 4   rape                 457 non-null    object
 5   robbery              457 non-null    object
 6   aggravated_assault   457 non-null    object
 7   property_crime       457 non-null    object
 8   burglary             457 non-null    object
 9   larceny              457 non-null    object
 10  car_theft            457 non-null    object
 11  arson                457 non-null    object
dtypes: object(12)
memory usage: 43.1+ KB


Why is there a missing value for 10/11 of the rows?

In [23]:
crime_by_city[crime_by_city.isnull().any(axis = 1)]

,city,population,violent_crime_count,murder_manslaughter,rape,robbery,aggravated_assault,property_crime,burglary,larceny,car_theft,arson
457,1 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This row is unnecessary, and hence gets dropped in the next line.

In [24]:
crime_by_city = crime_by_city[:-1]

All of the columns except **city** should have Dtype int64.

In [25]:
for col in ['population', 'violent_crime_count', 'murder_manslaughter', 'rape', 'robbery', 'aggravated_assault', 'property_crime', 'burglary', 'larceny', 'car_theft', 'arson']:
    crime_by_city[col] = crime_by_city[col].astype(int)

In [26]:
crime_by_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457 entries, 0 to 456
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 457 non-null    object
 1   population           457 non-null    int32 
 2   violent_crime_count  457 non-null    int32 
 3   murder_manslaughter  457 non-null    int32 
 4   rape                 457 non-null    int32 
 5   robbery              457 non-null    int32 
 6   aggravated_assault   457 non-null    int32 
 7   property_crime       457 non-null    int32 
 8   burglary             457 non-null    int32 
 9   larceny              457 non-null    int32 
 10  car_theft            457 non-null    int32 
 11  arson                457 non-null    int32 
dtypes: int32(11), object(1)
memory usage: 23.3+ KB


In [27]:
crime_by_city.head(10)

,city,population,violent_crime_count,murder_manslaughter,rape,robbery,aggravated_assault,property_crime,burglary,larceny,car_theft,arson
0,Adelanto,34491,276,1,20,42,213,459,136,209,114,14
1,Agoura Hills,20490,21,0,6,4,11,306,66,223,17,0
2,Alameda,78907,162,0,7,94,61,2579,218,1958,403,29
3,Albany,20083,40,0,8,21,11,685,105,534,46,1
4,Alhambra,84837,161,2,11,89,59,1749,259,1303,187,8
5,Aliso Viejo,52247,27,1,3,13,10,433,57,351,25,1
6,Alturas,2471,10,0,2,1,7,30,14,13,3,0
7,American Canyon,20452,53,0,7,7,39,454,59,368,27,3
8,Anaheim,353915,1120,8,141,396,575,8258,1123,5904,1231,44
9,Anderson,10545,61,1,5,12,43,363,63,208,92,1


Everything looks good! I will now engineer a column that gives the violent crime rate per 10,000 people for each city.

In [28]:
crime_by_city['vc_per_10k_ppl'] = crime_by_city['violent_crime_count'] / crime_by_city['population'] * 10000

Personally, I think it makes sense for the **vc_per_10k_ppl** column to be located immediately after the **population** and **violent_crime_count** columns.

In [29]:
crime_by_city_columns = list(crime_by_city.columns)

crime_by_city_columns.remove('vc_per_10k_ppl')
crime_by_city_columns.insert(3, 'vc_per_10k_ppl')

crime_by_city = crime_by_city[crime_by_city_columns]

In [30]:
crime_by_city.head(10)

,city,population,violent_crime_count,vc_per_10k_ppl,murder_manslaughter,rape,robbery,aggravated_assault,property_crime,burglary,larceny,car_theft,arson
0,Adelanto,34491,276,80.020875,1,20,42,213,459,136,209,114,14
1,Agoura Hills,20490,21,10.248902,0,6,4,11,306,66,223,17,0
2,Alameda,78907,162,20.530498,0,7,94,61,2579,218,1958,403,29
3,Albany,20083,40,19.917343,0,8,21,11,685,105,534,46,1
4,Alhambra,84837,161,18.977569,2,11,89,59,1749,259,1303,187,8
5,Aliso Viejo,52247,27,5.167761,1,3,13,10,433,57,351,25,1
6,Alturas,2471,10,40.469446,0,2,1,7,30,14,13,3,0
7,American Canyon,20452,53,25.914336,0,7,7,39,454,59,368,27,3
8,Anaheim,353915,1120,31.646017,8,141,396,575,8258,1123,5904,1231,44
9,Anderson,10545,61,57.847321,1,5,12,43,363,63,208,92,1


## Merging the Dataframes & (More) Feature Engineering

In [31]:
merged_df = pd.merge(crime_by_city[['city', 'population', 'vc_per_10k_ppl']], california_cities, on = 'city', how = 'inner')

In [32]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            429 non-null    object 
 1   population      429 non-null    int32  
 2   vc_per_10k_ppl  429 non-null    float64
 3   latitude        429 non-null    float64
 4   longitude       429 non-null    float64
dtypes: float64(3), int32(1), object(1)
memory usage: 15.2+ KB


Note that about 30 cities were dropped because of the merge.

In [33]:
merged_df = pd.merge(merged_df, ca_city_cmg_counts_df, on = 'city', how = 'outer')
merged_df['cmg_stores'] = merged_df['cmg_stores'].fillna(0)

In [34]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            466 non-null    object 
 1   population      429 non-null    float64
 2   vc_per_10k_ppl  429 non-null    float64
 3   latitude        429 non-null    float64
 4   longitude       429 non-null    float64
 5   cmg_stores      466 non-null    float64
dtypes: float64(5), object(1)
memory usage: 22.0+ KB


The above merge added 466 – 429 = 37 cities to the dataframe. Those cities should be dropped because they are missing too much data.

In [35]:
merged_df.dropna(inplace = True)
merged_df.reset_index(drop = True, inplace = True)

In [36]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            429 non-null    object 
 1   population      429 non-null    float64
 2   vc_per_10k_ppl  429 non-null    float64
 3   latitude        429 non-null    float64
 4   longitude       429 non-null    float64
 5   cmg_stores      429 non-null    float64
dtypes: float64(5), object(1)
memory usage: 20.2+ KB


It would be good to have a column that stores the addresses of the Chipotle stores for each city that has at least one. I will call this column **cmg_addresses**.

If a city has two or more Chipotle stores, the addresses will be separated using semicolons. If a city does not have any Chipotle stores, the value will be **N/A**.

In [37]:
combined_addresses_df = ca_chipotle_stores.groupby('city')['address'].apply(lambda x: '; '.join(x)).reset_index()

ca_chipotle_stores = pd.merge(ca_chipotle_stores, combined_addresses_df, on = 'city', suffixes = ('', '_all'))

ca_chipotle_stores = ca_chipotle_stores.rename(columns = {'address_all':'cmg_addresses'})

In [38]:
ca_city_cmg_addresses_df = ca_chipotle_stores[['city', 'cmg_addresses']]
ca_city_cmg_addresses_df = ca_city_cmg_addresses_df.drop_duplicates()

In [39]:
merged_df = pd.merge(merged_df, ca_city_cmg_addresses_df, on = 'city', how = 'outer')
merged_df['cmg_addresses'] = merged_df['cmg_addresses'].fillna('N/A')

In [40]:
merged_df.dropna(inplace = True)
merged_df.reset_index(drop = True, inplace = True)

In [41]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            429 non-null    object 
 1   population      429 non-null    float64
 2   vc_per_10k_ppl  429 non-null    float64
 3   latitude        429 non-null    float64
 4   longitude       429 non-null    float64
 5   cmg_stores      429 non-null    float64
 6   cmg_addresses   429 non-null    object 
dtypes: float64(5), object(2)
memory usage: 23.6+ KB


In [42]:
merged_df['population'] = merged_df['population'].astype(int)
merged_df['cmg_stores'] = merged_df['cmg_stores'].astype(int)

In [43]:
display(merged_df[merged_df['cmg_addresses'] == 'N/A'].sample(5, random_state = 105))
print()
display(merged_df[merged_df['cmg_addresses'] != 'N/A'].sample(5, random_state = 105))

,city,population,vc_per_10k_ppl,latitude,longitude,cmg_stores,cmg_addresses
193,Laguna Beach,23020,16.507385,33.542247,-117.783111,0,N/A
119,Etna,716,27.932961,41.456806,-122.894756,0,N/A
215,Los Altos,30716,6.836828,33.796331,-118.118119,0,N/A
426,Yreka,7527,37.199415,41.735419,-122.634472,0,N/A
167,Huntington Park,58181,78.719857,33.981681,-118.225072,0,N/A


,city,population,vc_per_10k_ppl,latitude,longitude,cmg_stores,cmg_addresses
33,Beverly Hills,34211,30.107275,34.073619,-118.400356,2,"244 S Beverly Dr Beverly Hills, CA 90212; 475 ..."
263,Oceanside,177129,40.196693,33.195869,-117.379483,3,"2611 Vista Way Ste A Oceanside, CA 92054; 4111..."
362,Selma,24983,34.023136,36.570783,-119.612075,1,"3120 Floral Ave 108A Selma, CA 93662"
271,Oxnard,211349,34.256136,34.197506,-119.177053,2,"1291 S Victoria Ave Oxnard, CA 93035; 369 W Es..."
295,Pomona,152776,61.527989,34.055228,-117.752306,1,"785 Rio Rancho Rd Ste 300 Pomona, CA 91766"


Wonderful! The **cmg_addresses** column is working exactly how it should!

I think it would be a great idea to have a column that ranks the cities from most violent to least! 

In the next line, the most violent city gets assigned the number 1, the second most violent the number 2, etc.

I also round the **vc_per_10k_ppl** to the nearest integer; in the event of a tie, the cities are assigned the same rank, and the ranks continue as if the rank was used once.

In [44]:
merged_df['vc_per_10k_ppl'] = merged_df['vc_per_10k_ppl'].astype(int)

vc_per_10k_ppl_sorted = merged_df.sort_values(by = 'vc_per_10k_ppl', ascending = False)

merged_df['vc_ranking'] = vc_per_10k_ppl_sorted['vc_per_10k_ppl'].rank(ascending = False, method = 'dense')
merged_df['vc_ranking'] = merged_df['vc_ranking'].astype(int)

In [45]:
merged_df_columns = list(merged_df.columns)

merged_df_columns.remove('vc_ranking')
merged_df_columns.insert(3, 'vc_ranking')

merged_df = merged_df[merged_df_columns]

What are the top 10 most violent cities in California?

In [46]:
display(merged_df.sort_values(by = 'vc_ranking').head(10))

,city,population,vc_per_10k_ppl,vc_ranking,latitude,longitude,cmg_stores,cmg_addresses
173,Industry,201,3582,1,34.019733,-117.958675,0,N/A
402,Vernon,112,2410,2,34.003903,-118.230072,0,N/A
344,Sand City,407,221,3,36.617183,-121.848286,1,"2140 California Ave Sand City, CA 93955"
177,Irwindale,1469,176,4,34.106953,-117.935342,0,N/A
252,Needles,5001,141,5,34.848061,-114.614133,0,N/A
379,Stockton,313604,139,6,37.957703,-121.290781,2,"10710 Trinity Pkwy Ste A Stockton, CA 95219; 4..."
115,Emeryville,12380,134,7,37.831317,-122.285247,0,N/A
382,Susanville,14878,131,8,40.416283,-120.653006,0,N/A
325,San Bernardino,216715,131,8,34.108344,-117.289764,1,"1092 E Hospitality Ln Ste B San Bernardino, CA..."
411,Weed,2673,127,9,41.422650,-122.386128,0,N/A


Wow... **Industry** and **Vernon** have extremely large **vc_per_10k_ppl** values. I think these values are extremely inflated due to both cities having <u>VERY</u> small populations.

I am curious to see the summary statistics of the **vc_per_10k_ppl** column.

In [47]:
merged_df['vc_per_10k_ppl'].describe()

count     429.000000
mean       48.337995
std       207.749793
min         0.000000
25%        16.000000
50%        28.000000
75%        44.000000
max      3582.000000
Name: vc_per_10k_ppl, dtype: float64

Now I am even more sure that **Industry** and **Vernon** have highly inaccurate **vc_per_10k_ppl** values. 

Let's replace make their **vc_per_10k_ppl** and **vc_ranking** values with –1, meaning they are unknown.

In [48]:
merged_df['vc_ranking'] = merged_df['vc_ranking'] - 2

In [49]:
merged_df.loc[(merged_df['city'] == 'Industry') | (merged_df['city'] == 'Vernon'), 'vc_per_10k_ppl'] = -1
merged_df.loc[(merged_df['city'] == 'Industry') | (merged_df['city'] == 'Vernon'), 'vc_ranking'] = -1

Did the code work correctly?

In [50]:
display(merged_df[(merged_df['city'] != 'Industry') & (merged_df['city'] != 'Vernon')].sort_values(by = 'vc_ranking').head(10))

,city,population,vc_per_10k_ppl,vc_ranking,latitude,longitude,cmg_stores,cmg_addresses
344,Sand City,407,221,1,36.617183,-121.848286,1,"2140 California Ave Sand City, CA 93955"
177,Irwindale,1469,176,2,34.106953,-117.935342,0,N/A
252,Needles,5001,141,3,34.848061,-114.614133,0,N/A
379,Stockton,313604,139,4,37.957703,-121.290781,2,"10710 Trinity Pkwy Ste A Stockton, CA 95219; 4..."
115,Emeryville,12380,134,5,37.831317,-122.285247,0,N/A
325,San Bernardino,216715,131,6,34.108344,-117.289764,1,"1092 E Hospitality Ln Ste B San Bernardino, CA..."
382,Susanville,14878,131,6,40.416283,-120.653006,0,N/A
261,Oakland,434036,127,7,37.804364,-122.271114,3,"3017 Broadway Oakland, CA 94611; 3271 Lakeshor..."
411,Weed,2673,127,7,41.422650,-122.386128,0,N/A
102,Dos Palos,5594,114,8,36.986058,-120.626572,0,N/A


Yes! I am especially happy to see that the **vc_ranking** values were correctly computed.

In [51]:
merged_df[(merged_df['city'] != 'Industry') & (merged_df['city'] != 'Vernon')]['vc_per_10k_ppl'].describe()

count    427.000000
mean      34.531616
std       27.699864
min        0.000000
25%       16.000000
50%       28.000000
75%       44.000000
max      221.000000
Name: vc_per_10k_ppl, dtype: float64

At what point does a city become a high outlier with respect to its **vc_per_10k_ppl** value?

In [52]:
print("A city's vc_per_10k_ppl value is a high outlier if it is greater than", 44 + 1.5*(44 - 16))

A city's vc_per_10k_ppl value is a high outlier if it is greater than 86.0


The next line creates a column named **high_violence**. 

If a city's **vc_per_10k_ppl** value is a high outlier, then the city is assign a **high_violence** value of 1, else 0 (except Industry and Vernon, which are assigned –1).

In [53]:
for i in range(len(merged_df)):
    if merged_df['vc_per_10k_ppl'][i] == -1:
        merged_df.loc[i, 'high_violence'] = -1
    elif merged_df['vc_per_10k_ppl'][i] > 86:
        merged_df.loc[i, 'high_violence'] = 1
    else:
        merged_df.loc[i, 'high_violence'] = 0

merged_df['high_violence'] = merged_df['high_violence'].astype(int)

In [54]:
merged_df['high_violence'].value_counts()

high_violence
 0    406
 1     21
-1      2
Name: count, dtype: int64

In [55]:
merged_df_columns = list(merged_df.columns)

merged_df_columns.remove('high_violence')
merged_df_columns.insert(4, 'high_violence')

merged_df = merged_df[merged_df_columns]

In [56]:
display(merged_df[merged_df['high_violence'] == 0].sample(5, random_state = 100))
print()
display(merged_df[merged_df['high_violence'] == 1].sample(5, random_state = 100))

,city,population,vc_per_10k_ppl,vc_ranking,high_violence,latitude,longitude,cmg_stores,cmg_addresses
246,Morro Bay,10624,19,77,0,35.365808,-120.849900,0,N/A
327,San Clemente,65018,13,83,0,33.426972,-117.611992,1,"806 Avenida Pico Ste D San Clemente, CA 92673"
398,Upland,77398,38,58,0,34.097511,-117.648389,2,"1053 E 19th St Ste A Upland, CA 91784; 1092 N ..."
338,San Marcos,98598,19,77,0,33.143372,-117.166144,1,"575 Grand Ave Ste 101 San Marcos, CA 92078"
171,Indian Wells,5500,23,73,0,33.717631,-116.340756,0,N/A


,city,population,vc_per_10k_ppl,vc_ranking,high_violence,latitude,longitude,cmg_stores,cmg_addresses
379,Stockton,313604,139,4,1,37.957703,-121.290781,2,"10710 Trinity Pkwy Ste A Stockton, CA 95219; 4..."
411,Weed,2673,127,7,1,41.422650,-122.386128,0,N/A
225,Marysville,12572,92,13,1,39.145725,-121.591356,0,N/A
382,Susanville,14878,131,6,1,40.416283,-120.653006,0,N/A
261,Oakland,434036,127,7,1,37.804364,-122.271114,3,"3017 Broadway Oakland, CA 94611; 3271 Lakeshor..."


## 10 Popular Attractions Dataframe

Ten of the most popular attractions in California are the following (listed in alphabetical order):

– Alcatraz Island

– Balboa Park

– Disneyland

– Hearst Castle

– Joshua Tree National Park

– Legoland California

– Malibu Beach

– Redwoods National Park

– Universal Studios Hollywood

– Yosemite National Park

It would be great to have the names of these attractions and their coordinates in a dataframe.

In [57]:
attractions_names = ['Alcatraz Island', 'Balboa Park', 'Disneyland', 'Hearst Castle', 'Joshua Tree National Park',
                     'Legoland California', 'Malibu Beach', 'Redwoods National Park', 'Universal Studios Hollywood', 'Yosemite National Park']

attractions_latitudes = [37.8265991, 32.7325629, 33.8120294, 35.6852218, 33.8875175, 33.1264746, 34.0318786, 37.8488593, 34.1373322, 37.6727756]
attractions_longitudes = [-122.4228001, -117.1472597, -117.9190063, -121.1679822, -115.8082581, -117.3113757, -118.6880654, -119.5570877, -118.3532224, -119.7282411]

In [58]:
attractions_df = pd.DataFrame({'attraction': attractions_names, 'latitude': attractions_latitudes, 'longitude': attractions_longitudes})

display(attractions_df)

,attraction,latitude,longitude
0,Alcatraz Island,37.826599,-122.422800
1,Balboa Park,32.732563,-117.147260
2,Disneyland,33.812029,-117.919006
3,Hearst Castle,35.685222,-121.167982
4,Joshua Tree National Park,33.887518,-115.808258
5,Legoland California,33.126475,-117.311376
6,Malibu Beach,34.031879,-118.688065
7,Redwoods National Park,37.848859,-119.557088
8,Universal Studios Hollywood,34.137332,-118.353222
9,Yosemite National Park,37.672776,-119.728241


**<u>NOTE</u>:** The coordinates were acquired from https://gps-coordinates.org/.

## To Be Continued

This project continues in the next notebook. It is in that notebook where one can find the model that estimates the shortest distance needed to travel by car to multiple cities in California and back. The model also computes which of the above ten attractions is located closest to one of the cities on a user's road trip itinerary.